In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import vaex as vx
import sys
from autocorrect import Speller
import re
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

tqdm.pandas()

In [2]:
#!pip3 install vaex
#!pip3 install pyspellchecker
#!pip install textblob
#!pip3 install autocorrect
#!pip3 install fuzzywuzzy
#!pip3 install python-Levenshtein

In [6]:
%%time
data = vx.open('history_small.csv')
data.__delitem__('weekday')
data.__delitem__('time')
data.__delitem__('Unnamed: 0')

CPU times: user 1.95 ms, sys: 1.11 ms, total: 3.06 ms
Wall time: 2.49 ms


In [4]:
# %%time
# data = pd.read_csv('history_small.csv')

CPU times: user 52.6 s, sys: 14.9 s, total: 1min 7s
Wall time: 1min 31s


In [17]:
data.dropnan(column_names=['UQ'])
data['UQ'] = data['UQ'].str.lower()

In [13]:
source = list(data['UQ'].unique())

In [ ]:
popularity = pd.read_csv('query_popularity.csv')

In [14]:
%%time
key_source = data['UQ'].values

In [18]:
word = {}
for i in tqdm(range(len(source))):
    x = source[i].split(' ')
    for j in x:
        if j in word.keys():
            word[j] +=1
        else:
            word[j] = 1

  0%|          | 20695/15148857 [00:00<00:48, 309290.43it/s]


AttributeError: 'NoneType' object has no attribute 'split'

In [20]:
source[i]

In [ ]:
double_words = {}
triple_words = {}
for i in tqdm(range(len(key_source))):
    x = key_source[i].split(' ')
    s = ''
    if (len(x))>=2:
        s = s.join(x[:1], ' ')
        if s in double_words.keys():
            double_words[s] += 1
        else:
            double_words[s] = 1
        if len(x)>2:
            s = ''
            s = s.join(x[:2], ' ')
            if s in triple_words.keys():
                triple_words[s] += 1
            else:
                triple_words[s] = 1

In [11]:
words = pd.DataFrame.from_dict(word, orient='index').reset_index()
words.rename({0:'num'}, axis=1, inplace=True)

In [12]:
len(word.keys())

3179730

In [ ]:
# spellru = Speller(lang='ru')
# spellen = Speller(lang='en')

# patternen = re.compile("[a-z]+")
# patternru = re.compile("[а-я]+")

# misswords = list(word.keys())
# correction = {}
# for x in tqdm(range(len(misswords))):
#     if len(misswords[x])<3:
#         correction[misswords[x]] = misswords[x]
#         continue
#     if patternen.fullmatch(misswords[x]):
#         correction[misswords[x]] = spellen(misswords[x])
#     elif patternru.fullmatch(misswords[x]):
#         correction[misswords[x]] = spellru(misswords[x])
#     else:
#         correction[misswords[x]] = misswords[x]

In [50]:
word_input = 'фильтратор'
word_list = list(word.keys())
temp = {}
for i in tqdm(range(len(word_list))):
    rat = fuzz.ratio(word_input, word_list[i])
    if rat>50:
        temp[word_list[i]] = rat
compare = pd.DataFrame.from_dict(temp, orient='index').reset_index()
compare.rename({0: 'simi'}, axis=1, inplace=True)
compare = compare.merge(words, how='left', on='index')
compare['metric'] = (0.01*compare['simi'])*np.log(compare['num'])
compare.sort_values(by='metric', ascending=False, inplace=True)
key_word = compare['index'].iloc[0]

100%|██████████| 3179730/3179730 [00:08<00:00, 381183.97it/s]


In [54]:
temp = {}
for i in tqdm(range(len(source))):
    if key_word in source[i]:
        x = source[i].split(' ')
        for j in x:
            if j in temp.keys():
                temp[j] +=1
            else:
                temp[j] = 1
most_used = pd.DataFrame.from_dict(temp, orient='index').reset_index()
most_used.rename({0:'use_num'}, axis=1, inplace=True)
most_used.sort_values(by='use_num', ascending=False, inplace=True)
most_used = most_used.head(100)
most_used = most_used.merge(words, how='left', on='index')

100%|██████████| 15148856/15148856 [00:05<00:00, 2771922.09it/s]
